In [1]:
import duckdb
con = duckdb.connect()

In [2]:
sql = """
Create Table if not exists device_status as (
    select ID, isBool, timestamp from device_status.csv
)
"""

con.sql(sql)

In [3]:
sql = """
select * from device_status
"""
con.sql(sql)

┌───────┬─────────┬─────────────────────┐
│  ID   │ isBool  │      timestamp      │
│ int64 │ boolean │      timestamp      │
├───────┼─────────┼─────────────────────┤
│     1 │ true    │ 2016-08-31 14:50:00 │
│     2 │ false   │ 2016-08-31 14:51:00 │
│     3 │ false   │ 2016-08-31 14:51:02 │
│     4 │ true    │ 2016-08-31 14:52:00 │
│     5 │ true    │ 2016-08-31 14:52:05 │
│     6 │ true    │ 2016-08-31 14:52:10 │
│     7 │ false   │ 2016-08-31 14:53:00 │
│     8 │ false   │ 2016-08-31 14:54:00 │
│     9 │ true    │ 2016-08-31 14:54:02 │
│    10 │ true    │ 2016-08-31 14:55:00 │
│    11 │ true    │ 2016-08-31 14:55:02 │
│    12 │ true    │ 2016-08-31 14:56:00 │
│    13 │ true    │ 2016-08-31 14:57:00 │
│    14 │ true    │ 2016-08-31 14:58:00 │
├───────┴─────────┴─────────────────────┤
│ 14 rows                     3 columns │
└───────────────────────────────────────┘

In [ ]:
sql = """
select 
        case when PreviousID is NULL then 0 else PreviousID end as PreviousID,
        PreviousStatus, PreviousTimestamp, 
        ID, isBool, timestamp
    from (
        SELECT 
            lag(ID) OVER validity AS PreviousID,
            lag(isBool) OVER validity AS PreviousStatus,
            lag(timestamp) OVER validity AS PreviousTimestamp,
            ID,
            isBool,
            timestamp
        FROM device_status
        WINDOW validity AS (ORDER BY ID, timestamp)
        ORDER BY timestamp
    ) where (PreviousStatus is NULL or isBool = PreviousStatus)
"""

con.sql(sql)

In [ ]:
sql = """
select isBool, 
        min(case when PreviousID is NULL then 0 else PreviousID end) as PreviousID, 
        min(PreviousTimestamp) as PreviousTimestamp, 
        max(ID) as ID, 
        max(timestamp) as timestamp
from (
    SELECT 
        ID,
        isBool,
        timestamp,
        lag(ID) OVER validity AS PreviousID,
        lag(isBool) OVER validity AS PreviousStatus,
        lag(timestamp) OVER validity AS PreviousTimestamp
    FROM device_status
    WINDOW validity AS (ORDER BY ID, timestamp)
    ORDER BY timestamp
) where (PreviousStatus is NULL or isBool = PreviousStatus)
group by PreviousID-ID, isBool
order by PreviousID
"""

con.sql(sql)

In [ ]:
sql = """
SELECT 
        ID,
        isBool,
        timestamp,
        lead(ID) OVER validity AS LeadID,
        lead(isBool) OVER validity AS LeadStatus,
        lead(timestamp) OVER validity AS LeadTimestamp
    FROM device_status
    WINDOW validity AS (ORDER BY ID, timestamp)
    ORDER BY timestamp
"""

con.sql(sql)

In [ ]:
sql = """
select ID, isBool, timestamp, leadID, leadStatus, leadTimestamp, epoch(leadTimestamp - timestamp) as Change 
from (
    SELECT 
        ID,
        isBool,
        timestamp,
        lead(ID) OVER validity AS LeadID,
        lead(isBool) OVER validity AS LeadStatus,
        lead(timestamp) OVER validity AS LeadTimestamp
    FROM device_status
    WINDOW validity AS (ORDER BY ID, timestamp)
    ORDER BY ID
) where LeadStatus is NULL or isBool = LeadStatus
"""

con.sql(sql)

In [ ]:
sql = """

SELECT
                id,
                isBool as term,
                timestamp as value,
                CASE
                    WHEN LAG(term) OVER (ORDER BY id) <> term OR LAG(term) OVER (ORDER BY id) IS NULL THEN 1
                    ELSE 0
                END AS term_change
            FROM
                device_status

"""

con.sql(sql)


In [ ]:
sql = """

SELECT
        id,
        term,
        value,
        SUM(term_change) OVER (ORDER BY id) AS term_group
    FROM
        (
            SELECT
                id,
                isBool as term,
                timestamp as value,
                CASE
                    WHEN LAG(term) OVER (ORDER BY id) <> term OR LAG(term) OVER (ORDER BY id) IS NULL THEN 1
                    ELSE 0
                END AS term_change
            FROM
                device_status
        ) t

"""

con.sql(sql)


In [4]:
sql = """

WITH numbered_data AS (
    SELECT
        id,
        term,
        value,
        SUM(term_change) OVER (ORDER BY id) AS term_group
    FROM
        (
            SELECT
                id,
                isBool as term,
                timestamp as value,
                CASE
                    WHEN LAG(term) OVER (ORDER BY id) <> term OR LAG(term) OVER (ORDER BY id) IS NULL THEN 1
                    ELSE 0
                END AS term_change
            FROM
                device_status
        ) t
)
SELECT
    term_group, min(id) as min_id, max(id) as max_id, term, min(value) as min_value, max(value) as max_value,
    epoch(max_value - min_value) as duration
FROM
    numbered_data curr
group by 
    term_group, term
order by
    term_group
"""

con.sql(sql)


┌────────────┬────────┬────────┬─────────┬─────────────────────┬─────────────────────┬──────────┐
│ term_group │ min_id │ max_id │  term   │      min_value      │      max_value      │ duration │
│   int128   │ int64  │ int64  │ boolean │      timestamp      │      timestamp      │  double  │
├────────────┼────────┼────────┼─────────┼─────────────────────┼─────────────────────┼──────────┤
│          1 │      1 │      1 │ true    │ 2016-08-31 14:50:00 │ 2016-08-31 14:50:00 │      0.0 │
│          2 │      2 │      3 │ false   │ 2016-08-31 14:51:00 │ 2016-08-31 14:51:02 │      2.0 │
│          3 │      4 │      6 │ true    │ 2016-08-31 14:52:00 │ 2016-08-31 14:52:10 │     10.0 │
│          4 │      7 │      8 │ false   │ 2016-08-31 14:53:00 │ 2016-08-31 14:54:00 │     60.0 │
│          5 │      9 │     14 │ true    │ 2016-08-31 14:54:02 │ 2016-08-31 14:58:00 │    238.0 │
└────────────┴────────┴────────┴─────────┴─────────────────────┴─────────────────────┴──────────┘